## Listing all tables in the database

In [6]:
import sqlite3
import pandas as pd

engine = sqlite3.connect('../kg_poc/datasets/eb_data_v1.db')
# select all tables from the database
sql = """
SELECT name FROM sqlite_master WHERE type='table'
"""
pd.read_sql_query(sql, engine)

,name
0,usage
1,species
2,sources
3,target
4,natural_product
5,compound
6,chembl_id_lookup
7,compound_records
8,target_dictionary
9,docs


## Selecting the usage and species name

In [8]:
import sqlite3
import pandas as pd

engine = sqlite3.connect('../kg_poc/datasets/eb_data_v1.db')
sql = """
SELECT u.usage_name, s.species_name as species_name
FROM usage u
JOIN species s
ON u.species_id = s.species_id
where u.usage_name = 'Analgesic'
"""
pd.read_sql_query(sql, engine)

,usage_name,species_name
0,Analgesic,Abies balsamea
1,Analgesic,Abies balsamea
2,Analgesic,Abies lasiocarpa
3,Analgesic,Acamptopappus sphaerocephalus
4,Analgesic,Acer rubrum
...,...,...
1247,Analgesic,Zigadenus venenosus
1248,Analgesic,Zingiber zerumbet
1249,Analgesic,Zinnia grandiflora
1250,Analgesic,Zizia aurea


## Linking the natural product to the usage and species name

In [9]:
sql = """
SELECT np.np_id, d.species_name, d.usage_name
FROM (
    SELECT u.usage_name, s.species_name as species_name
    FROM usage u
    JOIN species s
    ON u.species_id = s.species_id
    where u.usage_name = 'Analgesic'
) AS d
JOIN natural_product np
ON d.species_name = np.species_name
"""
pd.read_sql_query(sql, engine)

,np_id,species_name,usage_name
0,NPC100445,Abies balsamea,Analgesic
1,NPC150105,Abies balsamea,Analgesic
2,NPC185498,Abies balsamea,Analgesic
3,NPC217940,Abies balsamea,Analgesic
4,NPC217940,Abies balsamea,Analgesic
...,...,...,...
41121,NPC329908,Zinnia grandiflora,Analgesic
41122,NPC330022,Zinnia grandiflora,Analgesic
41123,NPC481151,Zinnia grandiflora,Analgesic
41124,NPC486694,Zinnia grandiflora,Analgesic


## Linking the compound to the natural product through the chembl_id

In [10]:
sql = """
SELECT c.chembl_id, c.np_id, c.pref_name as compound_name, d.species_name, d.usage_name
FROM (
    SELECT np.np_id, d.species_name, d.usage_name
    FROM (
        SELECT u.usage_name, s.species_name as species_name
        FROM usage u
        JOIN species s
        ON u.species_id = s.species_id
        where u.usage_name = 'Analgesic'
    ) AS d
    JOIN natural_product np
    ON d.species_name = np.species_name
) AS d
JOIN compound c
ON d.np_id = c.np_id
"""
pd.read_sql_query(sql, engine)

,chembl_id,np_id,compound_name,species_name,usage_name
0,CHEMBL3184678,NPC100445,1-Methyl-4-Prop-1-En-2-Ylcyclohexan-1-Ol,Abies balsamea,Analgesic
1,n.a.,NPC150105,YUKVPJYMFYPFEB-YKYSJNPSSA-N,Abies balsamea,Analgesic
2,CHEMBL44657,NPC185498,Etoposide,Abies balsamea,Analgesic
3,CHEMBL1967398,NPC217940,"Methyl (2E,6E)-9-(3,3-Dimethyloxiran-2-Yl)-3,7...",Abies balsamea,Analgesic
4,CHEMBL1967398,NPC217940,"Methyl (2E,6E)-9-(3,3-Dimethyloxiran-2-Yl)-3,7...",Abies balsamea,Analgesic
...,...,...,...,...,...
41121,n.a.,NPC329908,UGJQEYPUVSKREF-IHWZMDENSA-N,Zinnia grandiflora,Analgesic
41122,n.a.,NPC330022,YLSANYPALPIPQU-DYZKBYPLSA-N,Zinnia grandiflora,Analgesic
41123,n.a.,NPC481151,DCOPUUMXTXDBNB-UHFFFAOYSA-N,Zinnia grandiflora,Analgesic
41124,n.a.,NPC486694,UUASDOALLGVGAV-UQGZKMHZSA-N,Zinnia grandiflora,Analgesic


## Looking into CHembl to extract the compound records

In [11]:
sql = """
SELECT ch.chembl_id, ch.entity_type, ch.entity_id, d.species_name, d.usage_name
FROM (
    SELECT c.chembl_id, c.np_id, c.pref_name as compound_name, d.species_name, d.usage_name
    FROM (
        SELECT np.np_id, d.species_name, d.usage_name
        FROM (
            SELECT u.usage_name, s.species_name as species_name
            FROM usage u
            JOIN species s
            ON u.species_id = s.species_id
            where u.usage_name = 'Analgesic'
        ) AS d
        JOIN natural_product np
        ON d.species_name = np.species_name
    ) AS d
    JOIN compound c
    ON d.np_id = c.np_id
) AS d
JOIN CHEMBL_ID_LOOKUP ch
ON d.chembl_id = ch.chembl_id
"""
pd.read_sql_query(sql, engine)

,chembl_id,entity_type,entity_id,species_name,usage_name
0,CHEMBL3184678,COMPOUND,1673969,Abies balsamea,Analgesic
1,CHEMBL44657,COMPOUND,68607,Abies balsamea,Analgesic
2,CHEMBL1967398,COMPOUND,1288689,Abies balsamea,Analgesic
3,CHEMBL1967398,COMPOUND,1288689,Abies balsamea,Analgesic
4,CHEMBL2269327,COMPOUND,1486619,Abies balsamea,Analgesic
...,...,...,...,...,...
15632,CHEMBL245412,COMPOUND,407810,Zingiber zerumbet,Analgesic
15633,CHEMBL1240930,COMPOUND,700143,Zingiber zerumbet,Analgesic
15634,CHEMBL512339,COMPOUND,444681,Zingiber zerumbet,Analgesic
15635,CHEMBL512339,COMPOUND,444681,Zingiber zerumbet,Analgesic


In [12]:
sql = """
SELECT cr.molregno, cr.doc_id, cr.compound_name, d.species_name, d.usage_name
FROM (
    SELECT ch.chembl_id, ch.entity_type, ch.entity_id, d.species_name, d.usage_name
    FROM (
        SELECT c.chembl_id, c.np_id, c.pref_name as compound_name, d.species_name, d.usage_name
        FROM (
            SELECT np.np_id, d.species_name, d.usage_name
            FROM (
                SELECT u.usage_name, s.species_name as species_name
                FROM usage u
                JOIN species s
                ON u.species_id = s.species_id
                where u.usage_name = 'Analgesic'
            ) AS d
            JOIN natural_product np
            ON d.species_name = np.species_name
        ) AS d
        JOIN compound c
        ON d.np_id = c.np_id
    ) AS d
    JOIN CHEMBL_ID_LOOKUP ch
    ON d.chembl_id = ch.chembl_id
) AS d
JOIN COMPOUND_RECORDS cr
ON d.entity_id = cr.molregno
"""
pd.read_sql_query(sql, engine)

,molregno,doc_id,compound_name,species_name,usage_name
0,1673969,51887,SID144212695,Abies balsamea,Analgesic
1,68607,-1,ETOPOSIDE,Abies balsamea,Analgesic
2,68607,-1,ETOPOSIDE,Abies balsamea,Analgesic
3,68607,-1,ETOPOSIDE,Abies balsamea,Analgesic
4,68607,17494,Etoposide (VP-16),Abies balsamea,Analgesic
...,...,...,...,...,...
184210,78759,99699,Doxorubicin,Zinnia grandiflora,Analgesic
184211,78759,99725,Doxorubicin,Zinnia grandiflora,Analgesic
184212,78759,99762,Doxorubicin,Zinnia grandiflora,Analgesic
184213,78759,99843,Doxorubicin,Zinnia grandiflora,Analgesic


## Extract the targets related to the species name with Analgesic usage

In [13]:
sql = """
SELECT t.target_type, t.target_name, dr.species_name, dr.usage_name
FROM (
    SELECT np.np_id, d.species_name, d.usage_name
    FROM (
        SELECT u.usage_name, s.species_name as species_name
        FROM usage u
        JOIN species s
        ON u.species_id = s.species_id
        where u.usage_name = 'Analgesic'
    ) AS d
    JOIN natural_product np
    ON d.species_name = np.species_name
) AS dr
JOIN target t
ON dr.np_id = t.np_id
"""
# group by species_name and list the unique target type for each species name:
targets = pd.read_sql_query(sql, engine)
targets

,target_type,target_name,species_name,usage_name
0,Individual Protein,Peroxisome proliferator-activated receptor delta,Abies balsamea,Analgesic
1,NON-MOLECULAR,NON-PROTEIN TARGET,Abies balsamea,Analgesic
2,CELL-LINE,DLKP cell line,Abies balsamea,Analgesic
3,CELL-LINE,DLKP cell line,Abies balsamea,Analgesic
4,CELL-LINE,DLKP cell line,Abies balsamea,Analgesic
...,...,...,...,...
583073,Protein Family,Cyclooxygenase,Zinnia grandiflora,Analgesic
583074,Protein Family,Cyclooxygenase,Zinnia grandiflora,Analgesic
583075,SINGLE PROTEIN,Growth/differentiation factor 15,Zinnia grandiflora,Analgesic
583076,SINGLE PROTEIN,Replicase polyprotein 1ab,Zinnia grandiflora,Analgesic


In [12]:
targets.groupby('species_name')['target_type'].unique()

species_name
Abies balsamea                   [Individual Protein, Cell Line, NON-MOLECULAR,...
Abies lasiocarpa                 [CELL-LINE, Cell Line, Individual Protein, NON...
Acamptopappus sphaerocephalus                                   [Organism, Others]
Acer rubrum                      [Others, CELL-LINE, Cell Line, Individual Prot...
Acer saccharinum                 [CELL-LINE, Cell Line, Individual Protein, ORG...
                                                       ...                        
Xanthorhiza simplicissima        [Cell Line, Organism, Others, CELL-LINE, Indiv...
Zanthoxylum americanum           [Cell Line, Individual Protein, NON-MOLECULAR,...
Zea mays                         [Cell Line, Individual Protein, NON-MOLECULAR,...
Zingiber zerumbet                [Cell Line, Individual Protein, NON-MOLECULAR,...
Zinnia grandiflora               [CELL-LINE, Cell Line, Individual Protein, NON...
Name: target_type, Length: 262, dtype: object